In [1]:
import torch
from torch import nn


device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [2]:
from torchvision.models import resnet18


class PlateClf(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet18(pretrained=True)
        for param in self.backbone.parameters():
            param.requires_grad = False

        num_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Sequential(
            nn.Linear(num_features,128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128,1)
        )
    def forward(self,x):
        return self.backbone(x)



In [3]:
torch.manual_seed(42)
model = PlateClf()
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

C:\Workspace\endsieg\PyTorch\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Workspace\endsieg\PyTorch\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
from torchvision import datasets
from torchvision import transforms

data_transform = transforms.Compose([
    # Resize the images to 64x64
    transforms.Resize(size=(224, 224)),
    # Flip the images randomly on the horizontal
    transforms.RandomHorizontalFlip(p=0.5), # p = probability of flip, 0.5 = 50% chance
    # Turn the image into a torch.Tensor
    transforms.RandomRotation(10),  # лёгкое вращение
    transforms.ColorJitter(),       # яркость, контраст
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor() # this also converts all pixel values from 0 to 255 to be between 0.0 and 1.0
])

train_data = datasets.ImageFolder(root='./data/plates/train', transform=data_transform,target_transform=None)

from torch.utils.data import DataLoader

train_loader = DataLoader(dataset=train_data, batch_size=4, num_workers=6, shuffle=True)


In [5]:
n_epochs = 30 # do more, e.g. 30 since I am using data augmentation now
model.to(device)

for epoch in range(n_epochs):
    train_loss = 0
    for batch, (X, y) in enumerate(train_loader):
        model.train()
        X, y = X.to(device), y.float().unsqueeze(dim=1).to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_loss = train_loss/len(train_loader)

    print(f"Epoch {epoch+1}:")
    print(f"Train Loss: {train_loss:.4f}")


Epoch 1:
Train Loss: 0.7013
Epoch 2:
Train Loss: 0.6496
Epoch 3:
Train Loss: 0.6514
Epoch 4:
Train Loss: 0.6503
Epoch 5:
Train Loss: 0.6222
Epoch 6:
Train Loss: 0.6281
Epoch 7:
Train Loss: 0.6494
Epoch 8:
Train Loss: 0.5987
Epoch 9:
Train Loss: 0.5601
Epoch 10:
Train Loss: 0.5335
Epoch 11:
Train Loss: 0.5370
Epoch 12:
Train Loss: 0.5034
Epoch 13:
Train Loss: 0.5656
Epoch 14:
Train Loss: 0.5365
Epoch 15:
Train Loss: 0.5290
Epoch 16:
Train Loss: 0.5517
Epoch 17:
Train Loss: 0.4969
Epoch 18:
Train Loss: 0.4639
Epoch 19:
Train Loss: 0.4806
Epoch 20:
Train Loss: 0.5088
Epoch 21:
Train Loss: 0.4035
Epoch 22:
Train Loss: 0.4553
Epoch 23:
Train Loss: 0.4274
Epoch 24:
Train Loss: 0.4525
Epoch 25:
Train Loss: 0.4875
Epoch 26:
Train Loss: 0.3567
Epoch 27:
Train Loss: 0.3797


In [6]:
import os
from PIL import Image

test_transform = transforms.Compose([
    # Resize the images to 64x64
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor() # this also converts all pixel values from 0 to 255 to be between 0.0 and 1.0
])
ids = []
labels = []
for filename in os.listdir('./data/plates/test'):
    if filename.lower().endswith(('.png','.jpg','.jpeg')):
        img_path = os.path.join('./data/plates/test',filename)
        image = Image.open(img_path).convert('RGB')
        input_tensor = test_transform(image).unsqueeze(0).to(device)

        model.eval()
        with torch.inference_mode():
            out = model(input_tensor)
            out = torch.sigmoid(out)
            pred = (out>0.5).int()
        ids.append(filename[:4])
        label = train_data.classes[pred]
        labels.append(label)

In [7]:
import pandas as pd

df = pd.DataFrame()
df['label'] = labels
df['id'] = ids
df

,label,id
0,dirty,0000
1,dirty,0001
2,cleaned,0002
3,dirty,0003
4,dirty,0004
...,...,...
739,dirty,0739
740,dirty,0740
741,cleaned,0741
742,cleaned,0742


In [8]:
df.to_csv('plates_submission.csv',index=False)
